# API - Networks

This notebook illustrates the main features of pyNeVer for the creation of a neural network

## The _networks_ module

The module *networks* contains the classes __SequentialNetwork__ and __AcyclicNetwork__ to represent feed-forward and residual neural networks, respectively. Both subclass the abstract class __NeuralNetwork__ that provides base methods and utilities.

In [ ]:
from pynever.networks import NeuralNetwork, SequentialNetwork, AcyclicNetwork

# Create an empty FF network with identifier 'my_net' and input identifier 'X'
my_ff_net = SequentialNetwork('my_awesome_ff_net', 'X')

# Create an empty ResNet. Notice that the input identifiers are in a list to allow multiple inputs.
my_res_net = AcyclicNetwork('my_awesome_res_net', ['X_a', 'X_b'])

print(my_ff_net)
print(my_res_net)

print(isinstance(my_ff_net, NeuralNetwork))
print(isinstance(my_res_net, NeuralNetwork))

## The _nodes_ module

The module *nodes* contains the definition of NN layers as nodes in the computational graph. For the list of all supported layers, see [the documentation](http://www.neuralverification.org/pynever/API/1_Nodes.html). All nodes require a string identifier and the definition of the input dimension: the neural network object contains no information about this.

In [ ]:
from pynever import nodes
import torch

w = torch.Tensor([[1, 1], [-1, 1]])
b = torch.zeros(2)

# Create a fully connected layer with 2 inputs and 2 neurons.
# The input dimension in_dim is always a tuple
fc = nodes.FullyConnectedNode(identifier='fc', in_dim=(2,), out_features=2, weight=w, bias=b)

# Add it to the ff network
my_ff_net.append_node(fc)
# Let's add a ReLU layer now
my_ff_net.append_node(nodes.ReLUNode('relu', (2,)))

print(my_ff_net)

## Residual networks

For ResNets we provide a different method to add layers: *add_node* allows to specify the layer parents and, possibly, children

In [ ]:
rl = nodes.ReLUNode('relu', (2,))
fc_2 = nodes.FullyConnectedNode('fc_2', (2,), 2, weight=w, bias=b)
rl_2 = nodes.ReLUNode('relu_2', (2,))

my_res_net.add_node(fc)  # This is the first layer
my_res_net.add_node(rl, [fc])  # Layer rl follows fc
my_res_net.add_node(fc_2, [rl])  # Layer fc_2 follows rl
my_res_net.add_node(rl_2, [fc, fc_2])  # Layer rl_2 has a skip connection and has as parents both fc and fc_2

print(my_res_net)

# A few utility methods
print(f'Topological sort: {my_res_net.get_topological_order()}')
print(f'Parents of rl_2:  {my_res_net.get_parents(rl_2)}')
print(f'Children of fc:   {my_res_net.get_children(fc)}')
print(f'Leaves of the nn: {my_res_net.get_leaves()}')